In [1]:
from datetime import datetime as dt
import pandas as pd
import urllib

In [2]:
# get newest data url
today = dt.today()
print("Today's date:", today)
year=today.year
week=today.isocalendar().week
url=f'https://salmonprice.com/fishpool/servlets/newt/public/excel?type=details&from=2006-01&to={year}-{week}'

try:
 filepath = '../data/rawdata/Weekly_details.xls'
 urllib.request.urlretrieve(url, filepath)
 print('Downloaded successfully',url)
except:
 print('reminder : source need to update')

Today's date: 2023-12-20 21:03:54.904102
Downloaded successfully https://salmonprice.com/fishpool/servlets/newt/public/excel?type=details&from=2006-01&to=2023-51


In [3]:
# Read raw dara
path='../data/rawdata/Weekly_details.xls'
xls=pd.ExcelFile(path)
s = xls.sheet_names
s

['Fish Pool Index', 'Nasdaq 3-6', 'SSB Index', 'FPEBI 3-6', 'Farmers Index']

In [4]:
FishPool = xls.parse(s[0], skiprows=[0])
Nasdaq=xls.parse(s[1], skiprows=[0])
SSB = xls.parse(s[2], skiprows=[0])
FPEBI = xls.parse(s[3], skiprows=[0])
Farmer = xls.parse(s[4], skiprows=[0])

In [5]:
def time_convert(df):
 df = df.dropna(subset=['Year','Week'])
 df['Year']=df['Year'].astype('int')
 df['Week']=df['Week'].astype('int')
 df['t']=df['Year'].astype('str')+'-W'+df['Week'].astype('str')+ '-1'
 df['time']=[dt.strptime(t,"%Y-W%W-%w") for t in df['t']]
 return df

In [6]:
print(FishPool.columns)
print(Nasdaq.columns)

Index(['Year', 'Week', 'Farmers in FPI (FCA Oslo)',
       'Nasdaq 3-6 kg in FPI (FCA Oslo)', 'SSB in FPI (FCA Oslo)',
       'FPEBI 3-6kg', 'FPI EUR', 'FPI NOK', 'Weight Farmers',
       'Weight Nasdaq/NOS', 'Weight SSB', 'Weight FPEBI 3-6kg'],
      dtype='object')
Index(['Year', 'Week', '3-4 Kg', '4-5 Kg', '5-6 Kg', '3-4 Kg Weight',
       '4-5 Kg Weight', '5-6 Kg weight', 'Nasdaq 3-6 kg', 'Deduction Factor',
       'Nasdaq 3-6 kg in FPI (FCA Oslo)', 'Currency',
       'Nasdaq 3-6 kg in FPI (EUR)'],
      dtype='object')


In [7]:
FishPool = time_convert(FishPool)[['Year', 'Week','time','FPI EUR', 'FPI NOK']]
Currency = time_convert(SSB)[['time','Currency']]
Nasdaq = time_convert(Nasdaq)[['time','3-4 Kg','4-5 Kg','5-6 Kg']]

/var/folders/sb/09zclj8n6pdf03mfbjpflcz80000gn/T/ipykernel_4124/3743532486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year']=df['Year'].astype('int')
/var/folders/sb/09zclj8n6pdf03mfbjpflcz80000gn/T/ipykernel_4124/3743532486.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week']=df['Week'].astype('int')
/var/folders/sb/09zclj8n6pdf03mfbjpflcz80000gn/T/ipykernel_4124/3743532486.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [8]:
df = FishPool.merge(Currency,how='left')
df = df.merge(Nasdaq,how='left')

df['3-4 Kg EUR'] = (df['3-4 Kg']/df['Currency']).round(2)
df['4-5 Kg EUR'] = (df['4-5 Kg']/df['Currency']).round(2)
df['5-6 Kg EUR'] = (df['5-6 Kg']/df['Currency']).round(2)

df

,Year,Week,time,FPI EUR,FPI NOK,Currency,3-4 Kg,4-5 Kg,5-6 Kg,3-4 Kg EUR,4-5 Kg EUR,5-6 Kg EUR
0,2006,1,2006-01-02,3.11,24.71,7.9475,NaN,NaN,NaN,NaN,NaN,NaN
1,2006,2,2006-01-09,3.13,25.05,8.0145,NaN,NaN,NaN,NaN,NaN,NaN
2,2006,3,2006-01-16,3.32,26.89,8.0986,NaN,NaN,NaN,NaN,NaN,NaN
3,2006,4,2006-01-23,3.58,28.84,8.0567,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,5,2006-01-30,3.29,26.52,8.0702,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
946,2023,46,2023-11-13,6.51,77.03,11.8293,76.12,76.64,77.65,6.43,6.48,6.56
947,2023,47,2023-11-20,6.76,79.13,11.7075,77.79,79.09,79.75,6.64,6.76,6.81
948,2023,48,2023-11-27,7.17,83.86,11.7030,82.10,83.73,85.51,7.02,7.15,7.31
949,2023,49,2023-12-04,7.16,84.15,11.7482,82.65,83.74,85.72,7.04,7.13,7.30


In [9]:
# Save data
df.to_csv('../data/SalmonPrice.csv',index=False)

# Reference & Data Source
 - [Fish Pool](https://fishpool.eu/fpi-weekly-details/)